In [100]:
import iparser
import pandas as pd
import importlib
import pickle
import spacy
import tqdm
import utils 
from spacy.lang.en import English
from spacy.matcher import Matcher
nlp = spacy.load('en_core_web_sm')

## Топ профессий:

 

* Java разработчик/Software engineer
* Front-end разработчик/Web разработчик
* Продуктовый дизайнер
* Системный аналитик
* Архитектор систем
* Agile coach (под вопросом?)
* Data Scientist
* Data engineer
* Бизнес-аналитик
* Финансовый аналитик
* Кредитный аналитик
* Юрист (корпоративный, судебный)
* Маркетолог
 

 

# Используемый Подход 
* Парсер с определенной периодичностью запускается и соберает вакансии в базу
* Для анализа данные выгружаются из базы в датафрейм 


In [101]:
importlib.reload(iparser)
importlib.reload(utils)
pdict = utils.patterns_dict

In [57]:
vacancy_db = "vacancies.sqlite3"
import db_handler 
importlib.reload(db_handler)
db = db_handler.Db_handler(vacancy_db)
db.create_tab()

In [48]:
# for vaca in v.vacancies:
#     db.addvacancy(vaca)

In [58]:
v_ids = [x[0] for x in db.show_vacancy_ids()]

In [70]:
cols = ['vacancy_id','description','profession','country','city','reqs','skills','date']
vtab = pd.DataFrame(db.show_vacancies(), columns=cols)

In [71]:
vtab.shape

(49845, 8)

In [72]:
vtab.head()

,vacancy_id,description,profession,country,city,reqs,skills,date
0,/rc/clk?jk=8fb6adb20e0cc7c0&fccid=fe2d21eef233...,7+ years of relevant engineering experience3+ ...,Agile+coach,us,New+York,,,05-06-2020
1,/rc/clk?jk=66526d69241caf2c&fccid=48638adc9594...,"Innovation, collaboration, and success: at OnD...",Agile+coach,us,New+York,,,05-06-2020
2,/rc/clk?jk=9d8e229307c5f80e&fccid=ec00141efcac...,"At the NBA, we’re passionate about growing and...",Agile+coach,us,New+York,,,05-06-2020
3,/rc/clk?jk=42f82312c8e47d82&fccid=319b94ac5d94...,CLEAR's mission is to strengthen security and ...,Agile+coach,us,New+York,,,05-06-2020
4,/rc/clk?jk=437873289bb297a9&fccid=8009d67e7070...,Agile Coach Rock Star - hiring globally - (SO0...,Agile+coach,us,New+York,,,05-06-2020


In [73]:
vtab.profession.unique()

array(['Agile+coach', 'Java+Developer', 'Software+engineer',
       'Web+Developer', 'Front-end+Developer', 'Product+Designer',
       'System+analyst', 'System+architect', 'Data+Scientist',
       'Data+engineer', 'Business-analyst', 'Financial+analyst',
       'Credit+analyst', 'Corporate+lawyer', 'Judicial+Lawyer',
       'Marketing+Manager', 'Marketing+Specialist'], dtype=object)

In [74]:
vtab[vtab.profession == 'Data+Scientist'].city.unique()

array(['Dallas', 'Edinburgh', 'Texas', 'New+York', 'London', 'California'],
      dtype=object)

In [76]:
vtab[vtab.profession == 'Data+Scientist'].vacancy_id.nunique()

1833

In [80]:
vtab[vtab.profession == 'Data+Scientist'].groupby(['profession','city'])['vacancy_id'].count().reset_index()

,profession,city,vacancy_id
0,Data+Scientist,California,802
1,Data+Scientist,Dallas,91
2,Data+Scientist,Edinburgh,18
3,Data+Scientist,London,307
4,Data+Scientist,New+York,416
5,Data+Scientist,Texas,199


In [84]:
vacs_desc = vtab[vtab.profession == 'Data+Scientist']['description'].tolist()

In [86]:
nlp = English()

s = vacs_desc[0]
doc = nlp(s)
doc

Analyst I, Statistical (226 Days) -(20191119-019)

Description

Position Purpose:
The position’s main purpose is the development, production, and testing of Dallas ISD’s teacher and principal evaluation metrics, which includes the district’s value-added model. The main functions are statistical research, statistical computing, and documentation. Additional requirements include contributions to the production of E&A publications such as Campus Data Packets and School Profiles and the preliminary calculation of schools’ annual Texas state accountability statistics.

Basic Functions:
Develop value-added models, or revisions to existing models, for the purpose of evaluating the effectiveness of teacher performance
Perform extensive statistical computing using SPSS, SAS, Microsoft Access, and VBA
Develop and complete quality control procedures
Create, modify, and debug statistical computing programs in various languages/platforms
Create and maintain detailed documentation of processes and c

In [102]:
reqs = []
for s in tqdm.notebook.tqdm(vacs_desc):
    doc = nlp(s)
    res = requirements_extractor(doc)
    if res:
        if len(res) > 4:
#             print("_"*10)
#             print(res)
#             print(len(res))
            reqs.append(res)

In [103]:
len(reqs)

494

In [104]:
reqs

[requirements include contributions to the production of E&A publications such as Campus Data Packets and School Profiles and the preliminary calculation of schools’ annual Texas state accountability statistics.
 ,
 requirement. • Worked with different lines of business in consumer banking like mortgage, credit cards, loans and ready credit. • Preparing Business Requirement Document/Functional Requirements documents/Minor Development Documents (MDD) • Preparing the data enrichment rules • working with SMEs to get the sign offs for the changes • Data quality checks on feeds received • Good working knowledge in SQL, Data Analysis, Feed Analysis, Data Mapping, Data Reconciliation • Responsible for mapping the attributes required for Optima Retail Reporting with the country specific attributes for the regulatory reports such as CCAR14Q/FDIC/BASEL. • Working with business users across countries and products on the map-gap analysis performed to document the gaps. • Hands on experience in dat

In [194]:
reqs[5]

requirements of the role and your motivation behind applying.


In [166]:
requirements_extractor(doc)

In [254]:
s = vacancy_dict['Data Scientist'][30]
doc = nlp(s)
# doc

In [ ]:
matcher = Matcher(nlp.vocab)

pattern = [
    {'LEMMA':'requirement'},
    {'LEMMA':'skills'},
    {'LEMMA':'experience'},
]

In [261]:
for el in pdict.values():
    matcher.add('REQ_PATTERN', None, el)

In [150]:
# doc[502:]

In [116]:
c = 0
for el in doc[502:]:
    if len(el.text.split("\n")) > 2 and (el.i) < len(doc):
        print(doc[el.i+1], el.i+1)
        c = el.i + 1
        break


Accountabilities 600


In [124]:
doc[matches[0][1]:c]

Requirements:
A Finance/Economics/Business/Mathematics graduate
A demonstrated interest and knowledge in the financial markets
A proven record of outstanding achievement in academic and extracurricular activities
An independent thinker and decision maker who is able to contribute effectively to the team's success
Superior interpersonal and analytical skills
Ability to meet the challenges presented by a stressful, constantly changing work environment.
Ideally available to start asap – individual will be expected to go to Toronto for training 13th August for 2 wks.






In [97]:
matcher = Matcher(nlp.vocab)
for el in pdict.values():
    matcher.add('REQ_PATTERN', None, el)

In [98]:
def requirements_extractor(doc, matcher=matcher):
    endpos = None
    matches = matcher(doc)
        
    if len(matches) > 0 :
        startpos = matches[0][1]
        for el in doc[startpos:]:
            if len(el.text.split("\n")) > 2 and (el.i) < len(doc):
#                 print(doc[el.i+1], el.i+1)
                endpos = el.i + 1
                break
    if endpos:
        return doc[startpos:endpos]
    else:
        return None
    

In [99]:
reqs = []
for s in tqdm.notebook.tqdm(vacs_desc):
    doc = nlp(s)
    res = requirements_extractor(doc)
    if res:
        if len(res) > 4:
#             print("_"*10)
#             print(res)
#             print(len(res))
            reqs.append(res)
print(f"Удалось извлечь потенциальных скилов {len(reqs)} шт.")

ValueError: [E155] The pipeline needs to include a tagger in order to use Matcher or PhraseMatcher with the attributes POS, TAG, or LEMMA. Try using nlp() instead of nlp.make_doc() or list(nlp.pipe()) instead of list(nlp.tokenizer.pipe()).

In [161]:
bv = vacs_dict['Agile coach'][2]

In [162]:
docbv = nlp(bv)

In [163]:
docbv

Global Markets (Credit Products), High Yield Sector Strategist, VP, London, #152732
United Kingdom-London-London | Full-time (FT) | Global Markets | Job ID 152732
Credit Suisse is a leading global wealth manager with strong investment banking capabilities. Headquartered in Zurich, Switzerland, we have a global reach with operations in about 50 countries and employ more than 45,000 people from over 150 different nations. Embodying entrepreneurial spirit, Credit Suisse delivers holistic financial solutions to our clients, including innovative products and specially tailored advice. Striving for quality and excellence in our work, we recognize and reward extraordinary performance among our employees, provide wide-ranging training and development opportunities, and benefit from a diverse range of perspectives to create value for our clients, shareholders and communities. We are Credit Suisse.
We Offer

The Credit Suisse leverage finance trading desk is looking for a desk analyst at the VP/